In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# hack the paths, no package install needed
import sys
import os
repo_dir = '/media/user/repos/bulk-euclid-cutouts'
os.path.isdir(repo_dir)
sys.path.insert(0,repo_dir)

In [4]:
import logging
logging.basicConfig(level=logging.INFO)
import os

import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import pandas as pd

from bulk_euclid.utils import pipeline_utils
from bulk_euclid.external_targets import pipeline

/tmp/ipykernel_928/3545559080.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Authenticate

In [ ]:
# type in your COSMOS credentials
from astroquery.esa.euclid.core import Euclid
Euclid.login()

INFO:astroquery:Login to Euclid TAP server


INFO: Login to Euclid TAP server [astroquery.esa.euclid.core]


### Set Pipeline Options and Load External Targets

In [ ]:
# you can use this config and external target list for a minimal test on six targets within one tile...

cfg_dict = dict(
    # TODO change this path to wherever you want to save everything
    base_dir='/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version',
    name='external_targets_debug',
    release_name=None, # for no specific release or e.g. 'F-006' for one release 
    release_priority=['CALBLOCK_PV-005_R2', 'CALBLOCK_PV-005_R3', 'F-003_240321', 'F-003_240612' , 'F-006'],  # use the highest-index release first, if available
    bands=['VIS', 'NIR_Y'],
    auxillary_products=['MERPSF','MERRMS','MERBKG'],
    
    # these will be filled in automatically using on base_dir
    download_dir=None,
    tile_dir=None,
    cutout_dir=None,
    fits_dir=None,
    sanity_dir=None
    
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc
pipeline.create_folders(cfg)  # fill in those missing paths

external_targets = pd.DataFrame([
    {
      'id_str': 'a',
      'target_ra': 92.8371844681179,
      'target_dec': -47.8868920683699,
      'target_field_of_view': 20},
     {
      'id_str': 'b',
      'target_ra': 92.836691187603,
      'target_dec': -48.1117628801209,
      'target_field_of_view': 20},
     {
      'id_str': 'd',
      'target_ra': 92.835560566775,
      'target_dec': -47.978437467137,
      'target_field_of_view': 20},
     {
      'id_str': 'e',
      'target_ra': 92.8351734327103,
      'target_dec': -48.1054532354171,
      'target_field_of_view': 20},
     {
      'id_str': 'f',
      'target_ra': 92.8343976295701,
      'target_dec': -47.9993017111107,
      'target_field_of_view': 20}
    
])

In [ ]:
len(external_targets)

In [ ]:
external_targets

### For each target, find a tile that covers it

In [ ]:
targets_with_tiles = pipeline.get_matching_tiles(cfg, external_targets)
targets_with_tiles

In [ ]:
targets_with_tiles['release_name'].value_counts()

In [ ]:
targets_with_tiles = targets_with_tiles.sort_values('tile_index')[:1000]

The speed of the pipeline depends on the number of tiles to download. It takes about 30 seconds per tile, per band, per product. If you request two bands and four products (as above), each tile takes about 4 minutes.

In [ ]:
targets_with_tiles['tile_index'].nunique()

### For every tile, download the flux and auxillary (PSF, RMS, BKG) data and make cutouts around each target in that tile

In [ ]:
pipeline.make_cutouts(cfg, targets_with_tiles)